In [1]:
import pandas as pd
import json
import re
import numpy as np

In [2]:
lake = pd.read_csv("Resources/lake_info/lake_information.csv", converters={"Lake ID": str})
length = pd.read_csv("Resources/fish_length_info/fish_length.csv", converters={"lake_ID": str, "survey_ID": str})
catch = pd.read_csv("Resources/fish_catch_info/fish_catch.csv", converters={"lake_ID": str, "survey_ID": str})
was = pd.read_csv("Resources/was_info/water_access_information.csv")

In [3]:
#add methods
#sort and format year survey data then convert to datetime
def survey_date_cleaner(dfseries):
    dfseries = dfseries.copy()
    for _ in range(len(dfseries)):
        dfseries[_] = dfseries[_].replace("/","-")
        if matches := re.search(r"^(\d?\d{1})-(\d?\d{1})-(\d{4})$", dfseries[_], re.IGNORECASE):
            dfseries[_] = f"{matches.group(3)}-{matches.group(2)}-{matches.group(1)}"
        try:
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
        except ValueError:
            if matches := re.search(r"^(\d{4})-(\d?\d{1})-(\d?\d{1})$", dfseries[_], re.IGNORECASE):
                dfseries[_] = f"{matches.group(1)}-{matches.group(3)}-{matches.group(2)}"
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
    dfseries = pd.to_datetime(dfseries,format="%Y-%m-%d")
    return dfseries

#input CPUE column data series
#drop strings from cpue data series and convert to int(found some "? marks")
def cpue_cleaner(dfseries):
    i=0
    for _ in dfseries:
        try:
            np.float64(_)
            i = i + 1
        except ValueError:
            print(i, _,"valueerror")
            dfseries = dfseries.drop([i])
            i = i + 1
    dfseries = pd.to_numeric(dfseries)
    return dfseries

#input fish_count column data series
#returns fish_count column in dataframe as a list of individual integer measurements e.g. [0,0,1,1,1,2,3,3,4]
def fish_length_cleaner(dfseries):
    fish_lengths = []
    #scrub each row string and convert to list of lengths sampled
    for row in dfseries:
        #clean up string of list syntax
        sample = row.replace("]","").replace("[","").replace(" ","").split(",")
        #breakdown string into list of lengths sampled for row
        sample_lengths = []
        for n in range(len(sample)):
            if n % 2 == 1:
                fishlength = int(sample[n-1])
                fishcount = int(sample[n])
                for c in range(fishcount):
                    sample_lengths.append(fishlength)
        fish_lengths.append(sample_lengths)
    return fish_lengths 

#input quartile data series
#split lower quartile and convert to float
def lower_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[0]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list

#input quartile data series
#split upper quartile and convert to float
def upper_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[1]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list

In [ ]:
lake.info()

In [ ]:
lake = lake.drop_duplicates()
lake_coordinates = [json.loads(x) for x in lake["Lake Coordinates"]]
longitude = [x[0] for x in lake_coordinates]
latitude = [x[1] for x in lake_coordinates]
lake["Longitude"] = longitude
lake["Latitude"] = latitude
lake = lake.drop(columns=["Lake Coordinates"])
lake.head()

In [ ]:
lake.info()

In [ ]:
was.info()

In [ ]:
was = was.drop_duplicates()
site_coordinates = [json.loads(x) for x in was["Site Coordinates"]]
longitude = [x[0] for x in site_coordinates]
latitude = [x[1] for x in site_coordinates]
was["Longitude"] = longitude
was["Latitude"] = latitude
was = was.drop(columns=["Site Coordinates"])
was.head()

In [ ]:
was.info()

In [4]:
length.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100688 entries, 0 to 100687
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   lake_ID         100688 non-null  object
 1   species         100688 non-null  object
 2   fish_count      100688 non-null  object
 3   maximum_length  100688 non-null  int64 
 4   minimum_length  100688 non-null  int64 
 5   survey_ID       100688 non-null  object
 6   survey_date     100688 non-null  object
dtypes: int64(2), object(5)
memory usage: 5.4+ MB


In [6]:
length = length.drop_duplicates()
length["survey_date"] = survey_date_cleaner(length["survey_date"])

In [ ]:
length["fish_count"] = fish_length_cleaner(length["fish_count"])

In [13]:
length.sort_values("survey_date")

,lake_ID,species,fish_count,maximum_length,minimum_length,survey_ID,survey_date
61824,62006700,BIB,"[[0, 1]]",0,0,120236.0,1953-09-01
61823,62006700,GSF,"[[0, 1]]",0,0,120236.0,1953-09-01
61825,62006700,WTS,"[[0, 1]]",0,0,120236.0,1953-09-01
61822,62006700,BLG,"[[0, 1]]",0,0,120236.0,1953-09-01
61820,62006700,CAP,"[[0, 1]]",0,0,120236.0,1953-09-01
...,...,...,...,...,...,...,...
99471,73020002,WAE,"[[6, 26], [7, 21], [8, 18]]",8,6,1.6382134353095e+16,2021-10-14
51578,51006300,WAE,"[[10, 12], [6, 22], [7, 209], [8, 521], [9, 116]]",10,6,1.6312128751202e+16,2021-10-14
84215,03038100,LKS,"[[29, 1], [30, 1], [32, 1], [33, 1], [35, 1], ...",50,29,1.8332130535835e+16,2021-10-18
40957,38005800,RBT,"[[12, 5], [13, 5], [14, 1]]",14,12,1.763212912868e+16,2021-10-18


In [9]:
length.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100688 entries, 0 to 100687
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   lake_ID         100688 non-null  object        
 1   species         100688 non-null  object        
 2   fish_count      100688 non-null  object        
 3   maximum_length  100688 non-null  int64         
 4   minimum_length  100688 non-null  int64         
 5   survey_ID       100688 non-null  object        
 6   survey_date     100688 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 8.2+ MB


In [ ]:
catch.info()

In [ ]:
catch = catch.drop_duplicates()
catch["CPUE"] = cpue_cleaner(catch["CPUE"])
catch["survey_date"] = survey_date_cleaner(catch["survey_date"])
catch["count_lower_quartile"] = lower_quartile_cleaner(catch["quartileCount"])
catch["count_upper_quartile"] = upper_quartile_cleaner(catch["quartileCount"])
catch["weight_lower_quartile"] = lower_quartile_cleaner(catch["quartileWeight"])
catch["weight_upper_quartile"] = upper_quartile_cleaner(catch["quartileWeight"])
catch["gearCount"] = catch["gearCount"].astype("int")
catch = catch.drop(columns=["quartileCount", "quartileWeight"])

In [ ]:
catch.head()

In [ ]:
catch.info()

In [ ]:
lake.to_csv("Resources/lake_info/lake_information_clean.csv", index=False)
length.to_csv("Resources/fish_length_info/fish_length_clean.csv", index=False)
catch.to_csv("Resources/fish_catch_info/fish_catch_clean.csv", index=False)
was.to_csv("Resources/was_info/water_access_information_clean.csv", index=False)